# **Refresco de Pydantic** 🔧

Este notebook forma parte del curso de **IA Generativa** de la [Fundación GoodJob](https://www.fundaciongoodjob.org/).  
En él, los alumnos aprenderán a crear **esquemas de datos robustos** y **validación inteligente** que será la base para construir **agentes y sistemas de parseo avanzados** con LlamaIndex.

<p align="center">
  <img src="../notebooks/sources/pydantic.jpg" alt="Pydantic" style="width:100%; height:auto;"/>
</p>


## ¿Por qué Pydantic es crucial para IA Generativa? 🎯

En el mundo de la **IA Generativa**, trabajamos constantemente con datos **no estructurados** que necesitamos convertir en información **confiable y tipada**. Pydantic es nuestra herramienta secreta para:

- 🔍 **Parsear respuestas de LLMs** de forma estructurada y segura
- 🛡️ **Validar datos de APIs externas** antes de procesarlos
- 🤖 **Definir herramientas para agentes** con contratos claros
- 📄 **Extraer información de documentos** con garantías de calidad
- ⚡ **Normalizar datos heterogéneos** de múltiples fuentes

### El problema que resuelve Pydantic

**Sin Pydantic:**
```python
# ¡Código frágil y propenso a errores! 😱
user_data = api_response.json()
if 'email' in user_data and '@' in user_data['email']:
    email = user_data['email'].lower().strip()
    if 'age' in user_data:
        age = int(user_data['age'])  # ¿Y si no es un número?
        # ... más validaciones manuales
```

**Con Pydantic:**
```python
# ¡Código robusto y autodocumentado! ✨
@dataclass
class User(BaseModel):
    email: EmailStr
    age: Annotated[int, Field(ge=0, le=120)]
    
user = User.model_validate(api_response.json())  # ¡Validación automática!
```


## ¿Qué vamos a aprender hoy? 🎯

Al finalizar este notebook, serás capaz de:

- ✅ **Diseñar esquemas de datos robustos** con `BaseModel` y `Field`
- ✅ **Aplicar validación y normalización avanzada** con validadores personalizados
- ✅ **Serializar y deserializar** datos con control total sobre el formato
- ✅ **Crear uniones discriminadas** y tipos complejos para casos reales
- ✅ **Integrar Pydantic con APIs** para requests y responses seguros
- ✅ **Preparar el terreno para LlamaIndex** con salida estructurada


¡Empezamos esta aventura de datos estructurados! 🌟


## Configuración inicial y dependencias 🔧

Antes de empezar, vamos a instalar y configurar todo lo que necesitamos para trabajar con Pydantic v2 y sus herramientas complementarias.


In [26]:
# Install dependencies if needed
# !pip install "pydantic>=2.7" python-dateutil requests pytest email-validator

# Check if we have everything we need
import sys
from pathlib import Path

print("🔍 Verificando configuración del entorno...")

try:
    import pydantic
    print(f"✅ Pydantic v{pydantic.VERSION} instalado correctamente")
    
    import dateutil
    print("✅ python-dateutil disponible")
    
    import requests
    print("✅ requests disponible")
    
    from pydantic import EmailStr
    print("✅ email-validator disponible")
    
except ImportError as e:
    print(f"❌ Error: {e}")
    print("💡 Ejecuta: pip install \"pydantic>=2.7\" python-dateutil requests email-validator")

print("🎉 ¡Configuración inicial completada!")


🔍 Verificando configuración del entorno...
✅ Pydantic v2.11.7 instalado correctamente
✅ python-dateutil disponible
✅ requests disponible
✅ email-validator disponible
🎉 ¡Configuración inicial completada!


## 1. Fundamentos de Pydantic v2 📚

### 1.1 Tu primer modelo con BaseModel

Pydantic v2 introduce mejoras significativas en rendimiento y funcionalidad. Vamos a crear nuestro primer modelo robusto:


In [46]:
from datetime import datetime
from decimal import Decimal
from typing import Annotated, Optional
from pydantic import BaseModel, Field, EmailStr, ConfigDict

# Example 1: Basic customer model with modern Pydantic v2 syntax
class Customer(BaseModel):
    # v2 uses model_config instead of class Config
    model_config = ConfigDict(
        strict=True,      # Prevents unwanted type coercion
        extra='forbid'    # Rejects unexpected fields
    )
    
    # Using Annotated + Field for constraints (preferred over conint, etc.)
    id: Annotated[int, Field(ge=1, description="Customer unique identifier")]
    email: EmailStr
    signup_at: datetime
    credit: Annotated[Decimal, Field(ge=0, max_digits=12, decimal_places=2)] = Decimal("0.00")
    is_active: bool = True
    ref_code: Optional[str] = None

# Let's test our model
print("🧪 Testing Customer model...")

# Valid data
valid_customer_data = {
    "id": 123,
    "email": "user@example.com",
    "signup_at": datetime.fromisoformat("2024-01-15T10:30:00"),  # Pasar datetime, no string
    "credit": Decimal("100.50"),  # Pasar Decimal, no string
    "is_active": True
}

invalid_customer_data = {
    "id": 123,
    "email": "not-an-email",
    "signup_at": datetime.fromisoformat("2024-01-15T10:30:00"),  # Pasar datetime, no string
    "credit": Decimal("100.50"),  # Pasar Decimal, no string
    "is_active": True
}

try:
    customer = Customer.model_validate(valid_customer_data)
    print("✅ Customer created successfully!")
    print(f"   📧 Email: {customer.email}")
    print(f"   💰 Credit: €{customer.credit}")
    print(f"   📅 Signup: {customer.signup_at}")
    
    # Probamos con datos inválidos
    customer = Customer.model_validate(invalid_customer_data)
except Exception as e:
    print(f"❌ Validation error: {e}")

print("\n" + "="*50)


🧪 Testing Customer model...
✅ Customer created successfully!
   📧 Email: user@example.com
   💰 Credit: €100.50
   📅 Signup: 2024-01-15 10:30:00
❌ Validation error: 1 validation error for Customer
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='not-an-email', input_type=str]





### 1.3 Configuraciones esenciales con ConfigDict

Veamos las configuraciones más útiles para proyectos de IA:


In [ ]:
# Configuration examples for different use cases
from pydantic import BaseModel, ConfigDict

print("⚙️ Configuraciones esenciales de Pydantic...")

# Configuration 1: Strict mode for APIs (recommended)
class StrictAPIModel(BaseModel):
    model_config = ConfigDict(
        strict=True,           # No type coercion: "123" won't become int 123
        extra='forbid',        # Reject unexpected fields
        validate_assignment=True  # Re-validate when fields are modified
    )
    
    name: str
    age: int

# Configuration 2: Flexible mode for data processing
class FlexibleModel(BaseModel):
    model_config = ConfigDict(
        extra='allow',         # Allow extra fields (useful for exploratory analysis)
        validate_assignment=False,  # Don't re-validate on assignment (performance)
        str_strip_whitespace=True   # Auto-strip whitespace from strings
    )
    
    name: str
    age: int

# Configuration 3: ORM integration mode
class ORMModel(BaseModel):
    model_config = ConfigDict(
        from_attributes=True,  # Create models from ORM objects
        arbitrary_types_allowed=True,  # Allow custom types
        validate_assignment=True
    )
    
    name: str
    age: int

print("✅ Different configurations for different use cases:")
print("   🔒 StrictAPIModel: Perfect for API validation")
print("   🔄 FlexibleModel: Great for data exploration")  
print("   🗄️ ORMModel: Ideal for database integration")

# Let's test the differences
test_data = {"name": "  John Doe  ", "age": "25", "extra_field": "ignored"}

print("\n🧪 Testing configuration differences...")

try:
    # Strict mode will reject this
    strict_result = StrictAPIModel.model_validate(test_data)
    print("⚠️  Strict mode: Unexpectedly passed")
except Exception as e:
    print(f"✅ Strict mode: Correctly rejected - {str(e)}...")

try:
    # Flexible mode will accept this
    flexible_result = FlexibleModel.model_validate(test_data)
    print(f"✅ Flexible mode: Accepted with name='{flexible_result.name}'")
    print(f"   Extra fields: {flexible_result.__pydantic_extra__}")
except Exception as e:
    print(f"❌ Flexible mode: {e}")


⚙️ Configuraciones esenciales de Pydantic...
✅ Different configurations for different use cases:
   🔒 StrictAPIModel: Perfect for API validation
   🔄 FlexibleModel: Great for data exploration
   🗄️ ORMModel: Ideal for database integration

🧪 Testing configuration differences...
✅ Strict mode: Correctly rejected - 2 validation errors for StrictAPIModel
age
  Input...
✅ Flexible mode: Accepted with name='John Doe'
   Extra fields: {'extra_field': 'ignored'}


## 2. Validación y Normalización Avanzada 🔧

### 2.1 Validadores de Campo con `@field_validator`

Los validadores nos permiten **transformar y validar** datos de forma inteligente. Esto es especialmente útil cuando trabajamos con datos de múltiples fuentes o APIs externas.


In [30]:
from pydantic import field_validator, model_validator
from dateutil import parser as dtparse
import re
from decimal import Decimal, InvalidOperation, ROUND_HALF_UP, localcontext


print("🔧 Validadores avanzados para normalización de datos...")

class Transaction(BaseModel):
    model_config = ConfigDict(strict=True, extra='forbid')

    id: Annotated[str, Field(min_length=8)]
    amount: Annotated[Decimal, Field(gt=0, max_digits=14, decimal_places=2)]
    currency: Annotated[str, Field(pattern=r'^[A-Z]{3}$')]
    occurred_at: datetime
    description: Optional[str] = None

    # --- Normaliza moneda ---
    @field_validator('currency', mode='before')
    @classmethod
    def normalize_currency(cls, v: str) -> str:
        if not isinstance(v, str):
            raise ValueError("Currency must be a string")
        normalized = v.strip().upper()
        if len(normalized) != 3:
            raise ValueError("Currency must be exactly 3 characters")
        return normalized

    # --- Convierte amount a Decimal y limita a 2 decimales ---
    @field_validator('amount', mode='before')
    @classmethod
    def to_decimal_two_places(cls, v):
        """
        Acepta str/int/float/Decimal y devuelve Decimal cuantizado a 2 decimales.
        Evita construir Decimal desde float directamente cuando sea posible.
        """
        if isinstance(v, Decimal):
            d = v
        elif isinstance(v, (int,)):
            d = Decimal(v)
        elif isinstance(v, float):
            # Evitar binarios: convertir vía str
            d = Decimal(str(v))
        elif isinstance(v, str):
            try:
                d = Decimal(v.strip())
            except InvalidOperation:
                raise ValueError(f"Invalid decimal string: {v!r}")
        else:
            raise ValueError(f"Unsupported amount type: {type(v)}")

        # Cuantizar a 2 decimales con redondeo comercial
        with localcontext() as ctx:
            ctx.rounding = ROUND_HALF_UP
            d = d.quantize(Decimal("0.01"))

        # Reglas de max_digits/decimal_places ya las aplica Pydantic,
        # pero esta cuantización asegura formato estable.
        return d

    # --- Fechas flexibles ---
    @field_validator('occurred_at', mode='before')
    @classmethod
    def parse_datetime(cls, v):
        if isinstance(v, datetime):
            return v
        if isinstance(v, str):
            try:
                return dtparse.parse(v)
            except Exception:
                raise ValueError(f"Could not parse datetime: {v}")
        if isinstance(v, (int, float)):
            return datetime.fromtimestamp(v)
        raise ValueError(f"Unsupported datetime format: {type(v)}")

    # --- Limpieza de descripción ---
    @field_validator('description', mode='before')
    @classmethod
    def clean_description(cls, v):
        if v is None:
            return None
        if not isinstance(v, str):
            v = str(v)
        cleaned = re.sub(r'\s+', ' ', v.strip())
        return cleaned if cleaned else None

    # --- Reglas de negocio ---
    @model_validator(mode='after')
    def business_rules(self):
        # JPY sin decimales
        # (usa to_integral_value para evitar problemas de % con Decimal)
        if self.currency == 'JPY':
            if self.amount != self.amount.to_integral_value():
                raise ValueError('JPY transactions cannot have decimal amounts')

        # Importe grande requiere descripción
        if self.amount > Decimal("10000") and not self.description:
            raise ValueError('Large transactions (>10,000) require a description')

        return self

print("✅ Transaction model with advanced validation created!")

# --- Tests ---
test_cases = [
    {
        "name": "Normal case",
        "data": {
            "id": "TXN12345",
            "amount": "1200.50",
            "currency": " eur ",
            "occurred_at": "2024-01-15 14:30:00",
            "description": "  Payment   for   services  "
        }
    },
    {
        "name": "Unix timestamp",
        "data": {
            "id": "TXN67890",
            "amount": "500.00",
            "currency": "usd",
            "occurred_at": 1642248600,
            "description": "API payment"
        }
    },
    {
        "name": "JPY validation (should fail)",
        "data": {
            "id": "TXN11111",
            "amount": "1000.50",
            "currency": "jpy",
            "occurred_at": "2024-01-15T14:30:00"
        }
    }
]

for test_case in test_cases:
    print(f"\n🧪 Testing: {test_case['name']}")
    try:
        transaction = Transaction.model_validate(test_case['data'])
        print(f"✅ Success!")
        print(f"   💰 Amount: {transaction.amount} {transaction.currency}")
        print(f"   📅 Date: {transaction.occurred_at}")
        print(f"   📝 Description: '{transaction.description}'")
    except Exception as e:
        print(f"❌ Failed: {e}")


🔧 Validadores avanzados para normalización de datos...
✅ Transaction model with advanced validation created!

🧪 Testing: Normal case
✅ Success!
   💰 Amount: 1200.50 EUR
   📅 Date: 2024-01-15 14:30:00
   📝 Description: 'Payment for services'

🧪 Testing: Unix timestamp
✅ Success!
   💰 Amount: 500.00 USD
   📅 Date: 2022-01-15 13:10:00
   📝 Description: 'API payment'

🧪 Testing: JPY validation (should fail)
❌ Failed: 1 validation error for Transaction
  Value error, JPY transactions cannot have decimal amounts [type=value_error, input_value={'id': 'TXN11111', 'amoun...: '2024-01-15T14:30:00'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


### 2.2 [Modos de Validación](https://docs.pydantic.dev/2.8/concepts/validators/): 'before' vs 'after'

Los validadores pueden ejecutarse en diferentes momentos del proceso de validación. Entender cuándo usar cada modo es crucial para obtener el comportamiento deseado:

<div align="center" style="font-size:18px">

<table style="width:80%; text-align:center; border-collapse: collapse;" border="1">
  <tr>
    <th><b>Modo</b></th>
    <th><b>Cuándo se ejecuta</b></th>
    <th><b>Uso típico</b></th>
  </tr>
  <tr>
    <td><code>'before'</code></td>
    <td>Antes de la validación de tipo</td>
    <td>Normalización, limpieza de datos</td>
  </tr>
  <tr>
    <td><code>'after'</code></td>
    <td>Después de la validación de tipo</td>
    <td>Validación de reglas de negocio</td>
  </tr>
</table>

</div>


## 3. Serialización y Conversiones 📤

### 3.1 De Python a JSON y viceversa

Una de las fortalezas de Pydantic es su capacidad para serializar y deserializar datos de forma controlada. Esto es especialmente importante cuando trabajamos con APIs o necesitamos persistir datos.


In [31]:
from pydantic import TypeAdapter
import json

print("📤 Serialización y conversiones con Pydantic...")

# Let's use our Transaction model from before
transaction_data = {
    "id": "TXN12345",
    "amount": "1200.50",
    "currency": " eur ",
    "occurred_at": "2024-01-15 14:30:00",
    "description": "Payment for services"
}

# Create and validate transaction
tx = Transaction.model_validate(transaction_data)
print("✅ Transaction created and validated!")

# Serialization options
print("\n📋 Different serialization methods:")

# 1. Basic dictionary dump
basic_dict = tx.model_dump()
print(f"1️⃣ Basic dict: {basic_dict}")

# 2. Exclude None values
clean_dict = tx.model_dump(exclude_none=True)
print(f"2️⃣ Exclude None: {clean_dict}")

# 3. Include only specific fields
minimal_dict = tx.model_dump(include={'id', 'amount', 'currency'})
print(f"3️⃣ Include only: {minimal_dict}")

# 4. Exclude specific fields
no_desc_dict = tx.model_dump(exclude={'description'})
print(f"4️⃣ Exclude desc: {no_desc_dict}")

# 5. JSON string serialization
json_str = tx.model_dump_json(exclude_none=True)
print(f"5️⃣ JSON string: {json_str}")

print("\n" + "="*60)


📤 Serialización y conversiones con Pydantic...
✅ Transaction created and validated!

📋 Different serialization methods:
1️⃣ Basic dict: {'id': 'TXN12345', 'amount': Decimal('1200.50'), 'currency': 'EUR', 'occurred_at': datetime.datetime(2024, 1, 15, 14, 30), 'description': 'Payment for services'}
2️⃣ Exclude None: {'id': 'TXN12345', 'amount': Decimal('1200.50'), 'currency': 'EUR', 'occurred_at': datetime.datetime(2024, 1, 15, 14, 30), 'description': 'Payment for services'}
3️⃣ Include only: {'id': 'TXN12345', 'amount': Decimal('1200.50'), 'currency': 'EUR'}
4️⃣ Exclude desc: {'id': 'TXN12345', 'amount': Decimal('1200.50'), 'currency': 'EUR', 'occurred_at': datetime.datetime(2024, 1, 15, 14, 30)}
5️⃣ JSON string: {"id":"TXN12345","amount":"1200.50","currency":"EUR","occurred_at":"2024-01-15T14:30:00","description":"Payment for services"}



## 3.2 TypeAdapter: Validando listas y tipos complejos

`TypeAdapter` permite validar y serializar cualquier tipo anotado (listas, diccionarios, opcionales, uniones) sin necesidad de crear un `BaseModel`.

Es más flexible, evita modelos innecesarios, rápido en validaciones repetidas y mantiene la solidez del motor de Pydantic.


In [47]:
print("🔧 TypeAdapter para validar listas y tipos complejos...")

# Create TypeAdapters for different scenarios
transactions_adapter = TypeAdapter(list[Transaction])
currency_rates_adapter = TypeAdapter(dict[str, Decimal])
optional_transaction_adapter = TypeAdapter(Optional[Transaction])

# Test data: list of transactions
transactions_data = [
    {
        "id": "TXN001RT4",
        "amount": "100.00",
        "currency": "USD",
        "occurred_at": "2024-01-15T10:00:00"
    },
    {
        "id": "TXN002HG5", 
        "amount": "250.75",
        "currency": "EUR",
        "occurred_at": "2024-01-15T11:30:00",
        "description": "Online purchase"
    }
]

# Validate list of transactions
try:
    transactions = transactions_adapter.validate_python(transactions_data)
    print(f"✅ Validated {len(transactions)} transactions successfully!")
    
    for i, tx in enumerate(transactions, 1):
        print(f"   {i}. {tx.amount} {tx.currency} - {tx.id}")
        
except Exception as e:
    print(f"❌ Validation failed: {e}")

# Test currency rates dictionary
rates_data = {
    "USD": "1.0870",
    "GBP": "0.8650", 
    "JPY": "150.25"
}

try:
    rates = currency_rates_adapter.validate_python(rates_data)
    print(f"\n✅ Currency rates validated:")
    for currency, rate in rates.items():
        print(f"   {currency}: {rate}")
        
except Exception as e:
    print(f"❌ Rates validation failed: {e}")

# Serialize back to JSON
transactions_json = transactions_adapter.dump_json(transactions)
print(f"\n📤 Serialized transactions: {len(transactions_json)} bytes")

print("\n💡 TypeAdapter es perfecto para validar datos complejos de APIs!")


🔧 TypeAdapter para validar listas y tipos complejos...
✅ Validated 2 transactions successfully!
   1. 100.00 USD - TXN001RT4
   2. 250.75 EUR - TXN002HG5

✅ Currency rates validated:
   USD: 1.0870
   GBP: 0.8650
   JPY: 150.25

📤 Serialized transactions: 232 bytes

💡 TypeAdapter es perfecto para validar datos complejos de APIs!


## 4. Modelado Avanzado: Uniones Discriminadas y Campos Computados 🏗️

### 4.1 Uniones Discriminadas: Manejando tipos polimórficos

En el mundo real, a menudo necesitamos manejar datos que pueden tomar **distintas formas** (p. ej., diferentes métodos de pago, distintos tipos de eventos).  
Las **uniones discriminadas** permiten representar este polimorfismo de forma **segura y explícita**, guiando la validación según un **discriminante** (una clave común con un valor “tag” que indica la variante).


In [34]:
from typing import Literal, Union
from pydantic import computed_field, field_serializer, field_validator

print("🏗️ Modelado avanzado con uniones discriminadas...")

# Definir diferentes métodos de pago
class CardPayment(BaseModel):
    type: Literal['card']
    last4: Annotated[str, Field(pattern=r'^\d{4}$')]
    card_brand: Annotated[str, Field(min_length=1)]
    expiry_month: Annotated[int, Field(ge=1, le=12)]
    expiry_year: Annotated[int, Field(ge=2024, le=2030)]

class SepaPayment(BaseModel):
    type: Literal['sepa']
    iban: Annotated[str, Field(min_length=15, max_length=34)]
    bic: Optional[str] = None

class CryptoPayment(BaseModel):
    type: Literal['crypto']
    wallet_address: Annotated[str, Field(min_length=26, max_length=62)]
    currency: Annotated[str, Field(pattern=r'^[A-Z]{3,4}$')]  # BTC, ETH, etc.

# Crear unión discriminada
Payment = Annotated[
    Union[CardPayment, SepaPayment, CryptoPayment], 
    Field(discriminator='type')
]

# Modelo mejorado de transacción con método de pago
class EnhancedTransaction(BaseModel):
    model_config = ConfigDict(strict=True, extra='forbid')
    
    id: Annotated[str, Field(min_length=8)]
    amount: Annotated[Decimal, Field(gt=0, max_digits=14, decimal_places=2)]
    currency: Annotated[str, Field(pattern=r'^[A-Z]{3}$')]
    occurred_at: datetime
    payment_method: Payment
    description: Optional[str] = None

    # Normaliza la moneda a mayúsculas y sin espacios
    @field_validator('currency', mode='before')
    @classmethod
    def normalize_currency(cls, v: str) -> str:
        return v.strip().upper()
    
    # Convierte amount de str a Decimal si es necesario
    @field_validator('amount', mode='before')
    @classmethod
    def parse_amount(cls, v):
        # If already Decimal, return as is
        if isinstance(v, Decimal):
            return v
        try:
            return Decimal(str(v))
        except Exception:
            raise ValueError("No se pudo convertir el amount a Decimal")
    
    # Convierte occurred_at de str a datetime si es necesario
    @field_validator('occurred_at', mode='before')
    @classmethod
    def parse_occurred_at(cls, v):
        # If already datetime, return as is
        if isinstance(v, datetime):
            return v
        try:
            # Try parsing ISO format
            return datetime.fromisoformat(v)
        except Exception:
            raise ValueError("No se pudo convertir occurred_at a datetime")

print("✅ Enhanced transaction model with payment methods created!")

# Prueba de diferentes métodos de pago
test_payments = [
    {
        "name": "Card Payment",
        "data": {
            "id": "TXN_CARD_001",
            "amount": "99.99",
            "currency": "EUR",
            "occurred_at": "2024-01-15T14:30:00",
            "payment_method": {
                "type": "card",
                "last4": "1234",
                "card_brand": "Visa",
                "expiry_month": 12,
                "expiry_year": 2025
            },
            "description": "Online store purchase"
        }
    },
    {
        "name": "SEPA Payment", 
        "data": {
            "id": "TXN_SEPA_001",
            "amount": "1500.00",
            "currency": "EUR",
            "occurred_at": "2024-01-15T15:45:00",
            "payment_method": {
                "type": "sepa",
                "iban": "DE89370400440532013000",
                "bic": "COBADEFFXXX"
            },
            "description": "Invoice payment"
        }
    },
    {
        "name": "Crypto Payment",
        "data": {
            "id": "TXN_CRYPTO_001", 
            "amount": "0.05",
            "currency": "BTC",
            "occurred_at": "2024-01-15T16:20:00",
            "payment_method": {
                "type": "crypto",
                "wallet_address": "1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa",
                "currency": "BTC"
            }
        }
    }
]

for test_case in test_payments:
    print(f"\n🧪 Testing: {test_case['name']}")
    try:
        tx = EnhancedTransaction.model_validate(test_case['data'])
        print(f"✅ ¡Éxito! Tipo de pago: {tx.payment_method.type}")
        
        # Mostrar detalles específicos del pago según el tipo
        if tx.payment_method.type == 'card':
            print(f"   💳 Tarjeta terminada en {tx.payment_method.last4} ({tx.payment_method.card_brand})")
        elif tx.payment_method.type == 'sepa':
            print(f"   🏦 IBAN: {tx.payment_method.iban[:4]}...{tx.payment_method.iban[-4:]}")
        elif tx.payment_method.type == 'crypto':
            print(f"   ₿ Wallet: {tx.payment_method.wallet_address[:8]}...{tx.payment_method.wallet_address[-8:]}")
            
    except Exception as e:
        print(f"❌ Falló: {e}")


🏗️ Modelado avanzado con uniones discriminadas...
✅ Enhanced transaction model with payment methods created!

🧪 Testing: Card Payment
✅ ¡Éxito! Tipo de pago: card
   💳 Tarjeta terminada en 1234 (Visa)

🧪 Testing: SEPA Payment
✅ ¡Éxito! Tipo de pago: sepa
   🏦 IBAN: DE89...3000

🧪 Testing: Crypto Payment
✅ ¡Éxito! Tipo de pago: crypto
   ₿ Wallet: 1A1zP1eP...v7DivfNa


### 4.2 Campos Computados y Serializadores Personalizados

Los **campos computados** (`@computed_field`) permiten añadir valores **derivados** de otros campos, sin que formen parte de la entrada.  
Se calculan automáticamente y se pueden incluir en la serialización, lo que evita duplicar información y asegura coherencia.

In [37]:
print("🧮 Campos computados y serializadores personalizados...")

class Price(BaseModel):
    model_config = ConfigDict(strict=True)
    
    base: Annotated[Decimal, Field(ge=0, max_digits=10, decimal_places=2)]
    vat_rate: Annotated[Decimal, Field(ge=0, le=1)]  # 0.21 for 21% VAT
    discount_rate: Annotated[Decimal, Field(ge=0, le=1)] = Decimal('0.00')

    @computed_field
    @property
    def vat_amount(self) -> Decimal:
        """Calculate VAT amount"""
        return (self.base * self.vat_rate).quantize(Decimal('0.01'))

    @computed_field  
    @property
    def discount_amount(self) -> Decimal:
        """Calculate discount amount"""
        return (self.base * self.discount_rate).quantize(Decimal('0.01'))

    @computed_field
    @property
    def total(self) -> Decimal:
        """Calculate final total with VAT and discount"""
        subtotal = self.base - self.discount_amount
        return (subtotal + (subtotal * self.vat_rate)).quantize(Decimal('0.01'))

    @field_serializer('base', 'vat_amount', 'discount_amount', 'total')
    def serialize_decimals(self, v: Decimal) -> str:
        """Serialize Decimals as strings for safe JSON transport"""
        return str(v)

    @field_serializer('vat_rate', 'discount_rate')
    def serialize_rates(self, v: Decimal) -> float:
        """Serialize rates as percentages"""
        return float(v * 100)  # Convert 0.21 to 21.0

# Test the Price model
print("✅ Price model with computed fields created!")

price_data = {
    "base": Decimal("100.00"),
    "vat_rate": Decimal("0.21"),  # 21% VAT
    "discount_rate": Decimal("0.10")  # 10% discount
}

try:
    price = Price.model_validate(price_data)
    print(f"\n💰 Price calculation:")
    print(f"   Base price: €{price.base}")
    print(f"   Discount ({price.discount_rate * 100}%): -€{price.discount_amount}")
    print(f"   VAT ({price.vat_rate * 100}%): +€{price.vat_amount}")
    print(f"   Final total: €{price.total}")
    
    # Show serialization
    serialized = price.model_dump()
    print(f"\n📤 Serialized data: {serialized}")
    
    # JSON serialization
    json_data = price.model_dump_json()
    print(f"📄 JSON: {json_data}")
    
except Exception as e:
    print(f"❌ Error: {e}")

print("\n" + "="*60)


🧮 Campos computados y serializadores personalizados...
✅ Price model with computed fields created!

💰 Price calculation:
   Base price: €100.00
   Discount (10.00%): -€10.00
   VAT (21.00%): +€21.00
   Final total: €108.90

📤 Serialized data: {'base': '100.00', 'vat_rate': 21.0, 'discount_rate': 10.0, 'vat_amount': '21.00', 'discount_amount': '10.00', 'total': '108.90'}
📄 JSON: {"base":"100.00","vat_rate":21.0,"discount_rate":10.0,"vat_amount":"21.00","discount_amount":"10.00","total":"108.90"}



## 5. Integración con APIs: Request y Response Seguros 🌐

### 5.1 Validando datos de APIs externas

Cuando trabajamos con APIs externas, nunca sabemos exactamente qué datos vamos a recibir. Pydantic nos ayuda a crear **contratos claros** y **validación robusta** para nuestras integraciones.


In [39]:
from pydantic import HttpUrl
import requests
from typing import Dict

print("🌐 Integración segura con APIs externas...")

# Request models - what we send
class CurrencyRatesRequest(BaseModel):
    model_config = ConfigDict(strict=True)
    
    base: Annotated[str, Field(pattern=r'^[A-Z]{3}$')]
    symbols: list[Annotated[str, Field(pattern=r'^[A-Z]{3}$')]]
    
    @field_validator('base', 'symbols', mode='before')
    @classmethod
    def normalize_currencies(cls, v):
        if isinstance(v, str):
            return v.strip().upper()
        elif isinstance(v, list):
            return [item.strip().upper() for item in v]
        return v

# Response models - what we expect to receive
class CurrencyRatesResponse(BaseModel):
    model_config = ConfigDict(extra='ignore')  # Ignore extra fields from API
    
    base: str
    date: datetime
    rates: Dict[str, Decimal]
    
    @field_validator('date', mode='before')
    @classmethod
    def parse_api_date(cls, v):
        if isinstance(v, str):
            # Handle different date formats from APIs
            try:
                return dtparse.parse(v)
            except:
                # Fallback for YYYY-MM-DD format
                from datetime import datetime
                return datetime.strptime(v, '%Y-%m-%d')
        return v
    
    @field_validator('rates', mode='before')
    @classmethod
    def convert_rates(cls, v):
        # Convert string rates to Decimal for precision
        if isinstance(v, dict):
            return {k: Decimal(str(val)) for k, val in v.items()}
        return v

# API client function with Pydantic validation
def get_currency_rates(request: CurrencyRatesRequest, api_key: str = "demo") -> CurrencyRatesResponse:
    """
    Get currency rates from external API with full validation
    """
    # Simulate API call (in real scenario, you'd use actual API)
    # For demo, we'll create mock response data
    
    print(f"📡 Requesting rates for {request.base} -> {request.symbols}")
    
    # Mock API response (simulating what a real API might return)
    mock_api_response = {
        "base": request.base,
        "date": "2024-01-15",  # String date from API
        "rates": {
            "USD": "1.0870",  # String rates from API
            "GBP": "0.8650",
            "JPY": "150.25"
        },
        "timestamp": 1642248600,  # Extra field we don't need
        "success": True  # Another extra field
    }
    
    # Validate and return structured response
    try:
        validated_response = CurrencyRatesResponse.model_validate(mock_api_response)
        print("✅ API response validated successfully!")
        return validated_response
    except Exception as e:
        print(f"❌ API response validation failed: {e}")
        raise

# Test the API integration
print("🧪 Testing API integration...")

try:
    # Create request
    request = CurrencyRatesRequest(
        base="eur",  # Will be normalized to EUR
        symbols=[" usd ", "gbp", "jpy "]  # Will be normalized
    )
    
    print(f"📤 Request: {request.model_dump()}")
    
    # Make API call
    response = get_currency_rates(request)
    
    print(f"📥 Response validated:")
    print(f"   Base: {response.base}")
    print(f"   Date: {response.date}")
    print(f"   Rates:")
    for currency, rate in response.rates.items():
        print(f"     {currency}: {rate}")
        
    # Serialize for storage or further processing
    response_json = response.model_dump_json()
        
except Exception as e:
    print(f"❌ Integration failed: {e}")

print("\n💡 ¡Pydantic garantiza que los datos de APIs sean siempre confiables!")


🌐 Integración segura con APIs externas...
🧪 Testing API integration...
📤 Request: {'base': 'EUR', 'symbols': ['USD', 'GBP', 'JPY']}
📡 Requesting rates for EUR -> ['USD', 'GBP', 'JPY']
✅ API response validated successfully!
📥 Response validated:
   Base: EUR
   Date: 2024-01-15 00:00:00
   Rates:
     USD: 1.0870
     GBP: 0.8650
     JPY: 150.25

💡 ¡Pydantic garantiza que los datos de APIs sean siempre confiables!


## 6. Preparando el Terreno para LlamaIndex 🤖

### 6.1 Salida Estructurada de LLMs

Una de las aplicaciones más poderosas de Pydantic en IA Generativa es **forzar a los LLMs a generar salidas estructuradas**. Esto es fundamental para crear agentes confiables y sistemas de parseo robustos.


In [41]:
from enum import Enum
from typing import List

print("🤖 Modelos Pydantic para salida estructurada de LLMs...")

# Enums for controlled vocabularies (prevents hallucinated values)
class DocumentType(str, Enum):
    INVOICE = "invoice"
    CONTRACT = "contract"
    RECEIPT = "receipt"
    REPORT = "report"

class Priority(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    URGENT = "urgent"

class Sentiment(str, Enum):
    POSITIVE = "positive"
    NEGATIVE = "negative"
    NEUTRAL = "neutral"

# Models for structured LLM output
class ExtractedEntity(BaseModel):
    """Entity extracted from text"""
    text: str
    entity_type: Annotated[str, Field(min_length=1)]
    confidence: Annotated[float, Field(ge=0.0, le=1.0)]
    start_pos: Annotated[int, Field(ge=0)]
    end_pos: Annotated[int, Field(ge=0)]

class DocumentAnalysis(BaseModel):
    """Complete document analysis result"""
    model_config = ConfigDict(strict=True)
    
    document_type: DocumentType
    title: Annotated[str, Field(min_length=1, max_length=200)]
    summary: Annotated[str, Field(min_length=10, max_length=500)]
    key_entities: List[ExtractedEntity]
    sentiment: Sentiment
    priority: Priority
    confidence_score: Annotated[float, Field(ge=0.0, le=1.0)]
    
    # Business rules validation
    @model_validator(mode='after')
    def validate_business_rules(self):
        # High confidence should correlate with clear document type
        if self.confidence_score > 0.9 and self.document_type == DocumentType.REPORT:
            if len(self.key_entities) < 3:
                raise ValueError("High-confidence reports should have at least 3 key entities")
        
        # Urgent priority requires high confidence
        if self.priority == Priority.URGENT and self.confidence_score < 0.8:
            raise ValueError("Urgent priority requires confidence score >= 0.8")
            
        return self

class ToolInput(BaseModel):
    """Input schema for LlamaIndex tools"""
    model_config = ConfigDict(strict=True)
    
    query: Annotated[str, Field(min_length=1, description="Search query")]
    max_results: Annotated[int, Field(ge=1, le=100)] = 5
    filters: Optional[Dict[str, str]] = None

class ToolOutput(BaseModel):
    """Output schema for LlamaIndex tools"""
    model_config = ConfigDict(strict=True)
    
    results: List[str]
    total_found: Annotated[int, Field(ge=0)]
    execution_time_ms: Annotated[float, Field(ge=0)]
    
    @computed_field
    @property
    def success(self) -> bool:
        return len(self.results) > 0

print("✅ Structured output models created!")

# Simulate LLM output parsing
print("\n🧪 Testing structured LLM output...")

# Mock LLM response (what an LLM might generate)
mock_llm_output = {
    "document_type": "invoice",
    "title": "Monthly Service Invoice - January 2024",
    "summary": "Invoice for consulting services provided during January 2024, including project management and technical advisory.",
    "key_entities": [
        {
            "text": "€1,250.00",
            "entity_type": "amount",
            "confidence": 0.95,
            "start_pos": 45,
            "end_pos": 54
        },
        {
            "text": "January 2024",
            "entity_type": "date",
            "confidence": 0.92,
            "start_pos": 12,
            "end_pos": 24
        },
        {
            "text": "Acme Corp",
            "entity_type": "organization",
            "confidence": 0.88,
            "start_pos": 78,
            "end_pos": 87
        }
    ],
    "sentiment": "neutral",
    "priority": "medium",
    "confidence_score": 0.91
}

try:
    mock_llm_output_fixed = mock_llm_output.copy()
    mock_llm_output_fixed["document_type"] = DocumentType(mock_llm_output["document_type"])
    mock_llm_output_fixed["sentiment"] = Sentiment(mock_llm_output["sentiment"])
    mock_llm_output_fixed["priority"] = Priority(mock_llm_output["priority"])
    analysis = DocumentAnalysis.model_validate(mock_llm_output_fixed)
    print("✅ Salida del LLM validada correctamente!")
    print(f"   📄 Documento: {analysis.document_type.value}")
    print(f"   📝 Título: {analysis.title}")
    print(f"   🎯 Prioridad: {analysis.priority.value}")
    print(f"   📊 Confianza: {analysis.confidence_score:.2%}")
    print(f"   🔍 Entidades encontradas: {len(analysis.key_entities)}")
    
    for entity in analysis.key_entities:
        print(f"     - {entity.entity_type}: '{entity.text}' (conf: {entity.confidence:.2%})")
        
except Exception as e:
    print(f"❌ Falló la validación: {e}")

print("\n💡 Pydantic garantiza que los LLMs generen datos estructurados y válidos!")


🤖 Modelos Pydantic para salida estructurada de LLMs...
✅ Structured output models created!

🧪 Testing structured LLM output...
✅ Salida del LLM validada correctamente!
   📄 Documento: invoice
   📝 Título: Monthly Service Invoice - January 2024
   🎯 Prioridad: medium
   📊 Confianza: 91.00%
   🔍 Entidades encontradas: 3
     - amount: '€1,250.00' (conf: 95.00%)
     - date: 'January 2024' (conf: 92.00%)
     - organization: 'Acme Corp' (conf: 88.00%)

💡 Pydantic garantiza que los LLMs generen datos estructurados y válidos!


### 6.2 Creando un Dataset Sintético con Pydantic

Vamos a crear un generador de datos sintéticos que produzca información **realista y válida** usando nuestros modelos Pydantic.

In [42]:
import random
from datetime import datetime, timedelta
import uuid
import json

# Import LlamaIndex components for LLM-powered generation
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core.program import LLMTextCompletionProgram

print("🏭 Generador de datos sintéticos con LlamaIndex + Pydantic...")

# Configure LLM for structured output
llm = OpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,  # Add some creativity for varied synthetic data
    max_tokens=1000
)
Settings.llm = llm

class LLMDataGenerator:
    """Generador de datos sintéticos usando LLM con salida estructurada Pydantic"""
    
    def __init__(self):
        # Create LLM programs for each data type with structured output
        self.customer_program = LLMTextCompletionProgram.from_defaults(
            output_cls=Customer,
            prompt_template_str="""
            Generate realistic customer data for a SaaS platform. 
            Create a diverse customer profile with:
            - A unique ID between 1000-9999
            - A realistic email address
            - A signup date within the last year
            - A credit balance between 0-1000 EUR
            - Random active status (mostly active)
            - Optional referral code
            
            Make the data feel authentic and varied.
            """,
            llm=llm
        )
        
        self.transaction_program = LLMTextCompletionProgram.from_defaults(
            output_cls=EnhancedTransaction,
            prompt_template_str="""
            Generate a realistic financial transaction with payment method details.
            
            Requirements:
            - Transaction ID starting with "TXN_" followed by 8 uppercase chars
            - Amount appropriate for the currency (JPY without decimals, others with 2 decimals)
            - Currency: EUR, USD, GBP, or JPY
            - Recent timestamp (within last week)
            - One of three payment methods:
              * Card: with last4 digits, brand (Visa/Mastercard/American Express), expiry
              * SEPA: with valid-looking IBAN and BIC
              * Crypto: with wallet address and currency (BTC/ETH/ADA)
            - Optional realistic description
            
            Make it feel like a real business transaction.
            """,
            llm=llm
        )
        
        self.document_program = LLMTextCompletionProgram.from_defaults(
            output_cls=DocumentAnalysis,
            prompt_template_str="""
            Generate a realistic document analysis result as if processed by an AI system.
            
            Create analysis for document type: {doc_type}
            
            Requirements:
            - Document type from: invoice, contract, report, email, other
            - Descriptive title related to the document type
            - Brief summary of document contents
            - 2-4 extracted entities with realistic confidence scores (0.7-0.98)
            - Sentiment: positive, negative, or neutral
            - Priority: low, medium, or high
            - Overall confidence score (0.7-0.95)
            
            Make the entities relevant to the document type (amounts for invoices, dates, organizations, etc.)
            """,
            llm=llm
        )
    
    def generate_customer(self) -> Customer:
        """Generate a valid Customer instance using LLM"""
        try:
            return self.customer_program()
        except Exception as e:
            print(f"⚠️ LLM generation failed, using fallback: {e}")
            # Fallback to deterministic generation if LLM fails
            return Customer(
                id=random.randint(1000, 9999),
                email=f"user{random.randint(100, 999)}@example.com",
                signup_at=datetime.now() - timedelta(days=random.randint(1, 365)),
                credit=Decimal(str(round(random.uniform(0, 1000), 2))),
                is_active=True
            )
    
    def generate_transaction(self) -> EnhancedTransaction:
        """Generate a valid Transaction using LLM"""
        try:
            return self.transaction_program()
        except Exception as e:
            print(f"⚠️ LLM generation failed, using fallback: {e}")
            # Fallback implementation
            return EnhancedTransaction(
                id=f"TXN_{uuid.uuid4().hex[:8].upper()}",
                amount=Decimal("100.00"),
                currency="EUR",
                occurred_at=datetime.now() - timedelta(hours=1),
                payment_method={
                    "type": "card",
                    "last4": "1234",
                    "card_brand": "Visa",
                    "expiry_month": 12,
                    "expiry_year": 2025
                }
            )
    
    def generate_document_analysis(self, doc_type: str = None) -> DocumentAnalysis:
        """Generate a realistic document analysis using LLM"""
        if doc_type is None:
            doc_type = random.choice(["invoice", "contract", "report", "email", "other"])
            
        try:
            return self.document_program(doc_type=doc_type)
        except Exception as e:
            print(f"⚠️ LLM generation failed, using fallback: {e}")
            # Fallback implementation
            return DocumentAnalysis(
                document_type=DocumentType.INVOICE,
                title="Sample Invoice Analysis",
                summary="Automated analysis of business document",
                key_entities=[],
                sentiment=Sentiment.NEUTRAL,
                priority=Priority.MEDIUM,
                confidence_score=0.85
            )
        
# Create LLM-powered generator and produce synthetic data
generator = LLMDataGenerator()

print("🎲 Generating synthetic data with LLM...")

# Generate customers using LLM
print("\n🤖 Generating customers with AI...")
customers = []
for i in range(3):
    print(f"   Generating customer {i+1}/3...")
    customer = generator.generate_customer()
    customers.append(customer)

print(f"\n👥 Generated {len(customers)} customers:")
for i, customer in enumerate(customers, 1):
    print(f"   {i}. {customer.email} (ID: {customer.id}, Credit: €{customer.credit})")

# Generate transactions using LLM
print("\n🤖 Generating transactions with AI...")
transactions = []
for i in range(5):
    print(f"   Generating transaction {i+1}/5...")
    tx = generator.generate_transaction()
    transactions.append(tx)

print(f"\n💳 Generated {len(transactions)} transactions:")
for i, tx in enumerate(transactions, 1):
    print(f"   {i}. {tx.amount} {tx.currency} via {tx.payment_method.type} - {tx.id}")

# Generate document analyses using LLM
print("\n🤖 Generating document analyses with AI...")
analyses = []
doc_types = ["invoice", "contract", "report"]
for i, doc_type in enumerate(doc_types):
    print(f"   Generating {doc_type} analysis {i+1}/3...")
    analysis = generator.generate_document_analysis(doc_type)
    analyses.append(analysis)

print(f"\n📄 Generated {len(analyses)} document analyses:")
for i, analysis in enumerate(analyses, 1):
    print(f"   {i}. {analysis.document_type.value}: {analysis.title[:50]}... (conf: {analysis.confidence_score:.2%})")

# Serialize everything to JSON for storage/transmission
print(f"\n💾 Serializing all LLM-generated data to JSON...")

all_data = {
    "customers": [customer.model_dump() for customer in customers],
    "transactions": [tx.model_dump() for tx in transactions], 
    "analyses": [analysis.model_dump() for analysis in analyses],
    "generated_at": datetime.now().isoformat(),
    "total_records": len(customers) + len(transactions) + len(analyses),
    "generation_method": "LLM-powered with LlamaIndex + Pydantic validation"
}

json_output = json.dumps(all_data, indent=2, default=str)
print(f"✅ Generated {len(json_output)} bytes of valid JSON data!")
print(f"📊 Summary: {all_data['total_records']} records across 3 data types")

print("\n🎉 ¡Todos los datos generados por LLM son válidos según nuestros esquemas Pydantic!")
print("💡 El LLM crea datos más creativos y realistas, pero Pydantic garantiza su validez estructural.")


🏭 Generador de datos sintéticos con LlamaIndex + Pydantic...
🎲 Generating synthetic data with LLM...

🤖 Generating customers with AI...
   Generating customer 1/3...
   Generating customer 2/3...
   Generating customer 3/3...

👥 Generated 3 customers:
   1. johndoe@example.com (ID: 3876, Credit: €245.5)
   2. johndoe@example.com (ID: 4567, Credit: €250.75)
   3. john.doe@example.com (ID: 4567, Credit: €250.5)

🤖 Generating transactions with AI...
   Generating transaction 1/5...
   Generating transaction 2/5...
   Generating transaction 3/5...
   Generating transaction 4/5...
   Generating transaction 5/5...

💳 Generated 5 transactions:
   1. 1050.75 USD via card - TXN_ABCDEFGH
   2. 1234.56 USD via card - TXN_ABCDEFGH
   3. 1234.56 USD via card - TXN_ABCDEFGH
   4. 100.5 USD via card - TXN_ABCDEFGH
   5. 2500.75 USD via card - TXN_ABCDEFGH

🤖 Generating document analyses with AI...
   Generating invoice analysis 1/3...
   Generating contract analysis 2/3...
   Generating report analys

### 6.3 Ventajas del Generador con LLM + LlamaIndex

¡Excelente! Hemos transformado nuestro generador de datos sintéticos para usar un LLM con LlamaIndex. Veamos las ventajas de este enfoque:

**🎯 Beneficios del LLM + Pydantic:**
- **Creatividad controlada**: El LLM genera datos más diversos y realistas
- **Validación garantizada**: Pydantic asegura que todos los datos sean válidos
- **Escalabilidad**: Fácil agregar nuevos tipos de datos con prompts
- **Consistencia**: Los esquemas Pydantic mantienen la estructura

**💡 Casos de uso avanzados:**
- Generar datos de prueba para diferentes industrias
- Crear datasets sintéticos para entrenamiento
- Simular escenarios de negocio complejos
- Poblar bases de datos de desarrollo


In [ ]:
### 🚀 Consejos para Optimizar LLM + Pydantic

print("💡 Consejos para mejorar la generación con LLM...")

# 1. Configuración avanzada del LLM para mejor rendimiento
advanced_llm = OpenAI(
    model="gpt-4o-mini",  # Más económico y rápido para datos sintéticos
    temperature=0.8,       # Balance entre creatividad y consistencia
    max_tokens=500,        # Optimizado para respuestas estructuradas
    timeout=30.0           # Timeout para evitar bloqueos
)

# 2. Prompts más específicos mejoran la calidad
better_customer_prompt = """
Generate a realistic B2B SaaS customer profile for a company in {industry}.
The customer should reflect typical characteristics of {company_size} companies.

Requirements:
- ID: 4-digit number (1000-9999)
- Email: Professional domain matching industry
- Signup: Within last {days_range} days
- Credit: Appropriate for company size (startups: €50-500, enterprise: €1000-5000)
- Active status: 85% should be active
- Referral code: 30% chance, format REF + 4 digits

Industry context: {industry}
Company size: {company_size}
"""

# 3. Batch generation para mejor eficiencia
class BatchLLMGenerator:
    """Generador optimizado para lotes de datos"""
    
    def __init__(self):
        self.llm = advanced_llm
        
    def generate_batch_customers(self, count: int, industry: str = "technology") -> list[Customer]:
        """Generate multiple customers in one LLM call"""
        batch_prompt = f"""
        Generate {count} diverse customer profiles for {industry} industry.
        Return as JSON array with each customer having: id, email, signup_at, credit, is_active, ref_code.
        Ensure variety in company sizes, countries, and credit amounts.
        """
        
        # En una implementación real, usarías el LLM aquí
        # Por ahora, simulamos con fallback
        customers = []
        for i in range(count):
            customers.append(Customer(
                id=1000 + i,
                email=f"customer{i}@{industry.lower()}-corp.com",
                signup_at=datetime.now() - timedelta(days=random.randint(1, 365)),
                credit=Decimal(str(round(random.uniform(100, 2000), 2))),
                is_active=random.random() > 0.15  # 85% active
            ))
        return customers

# 4. Validación en tiempo real con retry logic
def generate_with_retry(generator_func, max_retries: int = 3):
    """Wrapper para reintentar generación si falla la validación"""
    for attempt in range(max_retries):
        try:
            return generator_func()
        except Exception as e:
            print(f"   ⚠️ Intento {attempt + 1} falló: {str(e)[:50]}...")
            if attempt == max_retries - 1:
                raise e
    return None



💡 Consejos para mejorar la generación con LLM...
✅ Configuración avanzada lista!
🎯 Estos patrones mejoran la calidad y eficiencia de la generación con LLM


## ¡Felicidades! 🎉 Has creado un generador de datos sintéticos inteligente  

### 🏆 ¿Qué has logrado hoy?

En este notebook transformaste por completo el sistema de generación de datos: pasaste de un **modelo aleatorio** a uno **impulsado por LLM con LlamaIndex**, manteniendo la **validación estricta de Pydantic**. Lograste combinar creatividad y realismo en los datos con control estructural, además de añadir mecanismos de fallback para garantizar robustez y escalabilidad en la incorporación de nuevos tipos de datos.  

### 🚀 ¿Por qué es esto revolucionario?

Porque une lo mejor de dos mundos: la **capacidad generativa de los LLMs**, que producen datos diversos y realistas, y la **seguridad de Pydantic**, que asegura que cada ejemplo cumpla los esquemas definidos. Esto no solo mejora la calidad de los datos, sino que también habilita su uso en entornos de prueba exigentes, permite reutilizar respuestas mediante cache, escalar a distintos dominios con prompts específicos y medir la calidad de manera objetiva.  


### 🛣️ ¿Qué sigue?

En los próximos notebooks aprenderás:

- ✅ ✨ **Introducción a LLMs, agentes y tools**
- ✅ 🤖 **Sistemas multi-agente y ChromaDB**
- ✅ 🏗️ **Validación con Pydantic**
- ⬜ 📄 **Parseo avanzado de documentos**
- ⬜ 🚀 **Despliegue en producción**

**¡El futuro de los datos sintéticos está en la fusión de IA generativa con validación estructural! 🌟**  
